In [1]:
#math and linear algebra stuff
import numpy as np
import numpy.linalg as la

#plots
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (15.0, 15.0)
#mpl.rc('text', usetex = True)
import matplotlib.pyplot as plt
%matplotlib inline

# Chebyshev Polynomial Acceleration

This notebook intend to study the roots of the so called "Chebyshev Polynomial Acceleration" that encountered a large success in the field of first order optimization method acceleration.